## Part 1 - Cosmic Ray Generation

In order to simulate the response of the detector, we need a simulation of the incoming particles. Here we will consider cosmic rays, which we will simulate using a Monte Carlo method.

At the Earth's surface, cosmic rays are essential all muons.  To a good approximation, they have a distribution in zenith angle $\theta$ which is proportional to ${\rm cos}^2(\theta)$, and are uniform in polar angle $\phi$. We can assume they are uniformly distributed in the horizontal plane.

If the energy of the cosmic muon needs to be modelled, the distribution can be very crudely approximated in units of GeV/C by a log-normal distribution with $\mu=6.55$ and $\sigma=1.8$.  If the charge needs to be modelled, cosmic rays at the Earth's surface comprise approximately 30% more $\mu^+$ than $\mu^-$.

### Writing the code

Write a function that will randomly generate a cosmic ray muon each time it is called.  The function should return at least a direction in 3D, given the above distributions, as well as a starting position on a user definable horizontal plane.  Show that your function produces the desired distributions, at least qualitatively, by calling it for a large number of trials.

In [ ]:
import scipy
import numpy as np
import scipy.stats
import scipy.sparse
from scipy.stats import moyal
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.sparse.linalg import factorized

class Particle():
    """
    Main class containing the methods  required for the  generation of 
    muons originating from  a plane of much larger dimensions than the
    drift chamber. 
    """
    def __init__(self, mean_p=6.55, std_p=1.8):
        """
        This method defines the mean and  standard 
        deviation of the lognormal distribution of
        the momentum of the generated muons.
        """
             
        self.mean_p = mean_p       # Mean momentum of cosmic ray muons. 
        self.std_p = std_p         # The stardard deviation.    
        self.e_charge_nat = 0.303  # Electron charge in natural units. 
    
    def ChargeGenerator(self):
        """
        This method is responsible for assigning 
        charges to the generated muons randomly.
        """
        
        x = np.random.random()
        
        if x< 0.5652173913:            # Allows for a charge ratio of 1.3:1 between positive and negative muons.
            return self.e_charge_nat   # Positive charge of muon in natural units.
        else:
            return -self.e_charge_nat  # Negative charge of muon in natural units.

    def MomentumGenerator(self):
        """
        This method assigns a random momentum to the generated muons.
        The momentum distribution is log-normal , with mean 6.55 and 
        standard deviation 1.8.
        """
        
        self.mu = 6.55    # Mean for the LogNormal Distribution of momentum.
        self.sigma = 1.8  # The standard deviation.
         
        return np.random.lognormal(self.mu, self.sigma)  # Returns LogNormal Distribution of momentum with mu and sigma.
    
    def ZenithAngle(self):
        """
        This method generates values for the Zenith angles distributed proportional
        to cos^2(θ) using the Accept/Reject method since there is no inverse.
        """
        
        while True:
            theta = (np.pi/2)*(np.random.random())
            y = np.random.random()
            
            if y < (np.cos(theta))**2:
                return theta
            else:
                continue

    def AzimuthialAngle(self):
        """
        This method generates values for the Azimuthial angles distributed uniformly.
        """
                       
        return 2*np.pi*np.random.random()  # Uniform Distribution. 

    def Plane_Entry_Location(self,x_length, y_length):
        """
        Simulates the starting coordinates of the generated muons from the plane
        above the drift chamber.
        """
        
        entry_x = x_length*np.random.random()  # x Coordinate of horizontal plane above chamber.
        entry_y = y_length*np.random.random()  # y Coordinate of horizontal plane above chamber.
        
        return [entry_x, entry_y]  # (x,y) Coordinates of horizontal plane above chamber.          

    def Vector_Direction(self, theta, phi):
        """
        Utilises the Zenith and Azimuthial angle to return the direction of a vector
        in Cartesian Coordinates.
        """
         
        v = np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), -np.cos(theta)])
        return v
    
    def Main_Generator(self,plane_x, plane_y, mass=0.1057):
        """
        Utilises all of the previously defined methods to collectively generate 
        all of the required variables in order to simulate the generation of
        the muons as well as their entry in drift chamber. 
        """
        
        self.mass = mass            
        phi = self.AzimuthialAngle()
        theta = self.ZenithAngle()
        self.momentum_mag = self.MomentumGenerator()                
        self.momentum_dir = self.Vector_Direction(theta, phi)    
        self.charge = self.ChargeGenerator()
        self.init_pos = self.Plane_Entry_Location(plane_x, plane_y)
        self.energy = np.sqrt(self.momentum_mag**2 + self.mass**2)

In [ ]:
Muon = Particle() # Creating an instance of the Particle(). 

def Residual_Analysis(residual, fit_tested):
    """
    Implemented to print the mean and standard deviation of the 
    residuals distribution of any targeted dataset.  
    """

    print('\n' +fit_tested+ ':') 

    mean = np.mean(residual)
    std = np.std(residual)
    
    print('Residual mean = ', "{:.2e}".format(mean))
    print('Residual std dev = ', "{:.2e}".format(std)) 

def Charge_Ratio_Validation(no_points=100000):
    """
    Validates the fact given by the assignment that there are approximately 
    30% more positive muons than negative muons in the cosmic rays at the 
    Earths surface.
    """
    
    charge_vals   = [Muon.ChargeGenerator() for i in range(no_points)]

    pos_muons = charge_vals.count(Muon.e_charge_nat)    
    neg_muons = charge_vals.count(-Muon.e_charge_nat) 
    
    ratio = pos_muons/neg_muons          
    excess_percent = (ratio-1)*100       

    print('Number of Main_Generatord muons is: ', no_points)
    print('\t Positive muons: ', pos_muons)
    print('\t Negative muons: ', neg_muons)
    print('Excess of positive muons: ', "{:.2f}". format(excess_percent),'%\n')       
        
def Plotter_to_Verify(no_points=100000, no_bins=100):
    """
    This large function visualises and validates the distribution of the generated variables.
    The lognormal distribution of the muons momentum, the cos^2(θ) distribution of Zenith 
    angles and the uniform distribution of the Azimuthial angles are all checked by
    plotting the relevant histograms, fitting a line of best fit and presenting
    the histograms of residuals. 
    
    Moreover a Gaussian Kernel Density Estimate for the scatter plot of muons on the 
    large plane above the drift chamber is plotted which quantitatively demonstrates
    the density of muons across the 2-D plane.
    """
    
    init_xy_vals  = [Muon.Plane_Entry_Location(2,2) for i in range(int(no_points/2))]
    momentum_vals = [Muon.MomentumGenerator() for i in range(no_points)]
    theta_vals    = [Muon.ZenithAngle() for i in range(no_points)]
    phi_vals      = [Muon.AzimuthialAngle() for i in range(no_points)]

    
    fig, axs = plt.subplots(4, 2, figsize=(20, 22), dpi=150)

    #***Scatter plot of the starting position of the muons on the plane above the chamber.***#
    
    init_x_vals = [i[0] for i in init_xy_vals]  # Used for the Gaussian KDE.
    init_y_vals = [i[1] for i in init_xy_vals]  # Used for the Gaussian KDE.
    
    axs[0, 0].scatter(init_x_vals, init_y_vals, s=10, color='green', alpha=0.5, linewidth=0.5)
    axs[0, 0].set(title='Scatter plot of muon positions on the plane.', xlabel='X Dimension (m)', ylabel='Y Dimension (m)')
    
    xmin = np.amin(init_x_vals)  # Finds minimum x value.
    xmax = np.amax(init_x_vals)  # Finds maximum x value.
    ymin = np.amin(init_y_vals)  # Finds minimum y value.
    ymax = np.amax(init_y_vals)  # Finds maximum y value.
    
    X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]  # Creates a meshgird for the coordinate system.
    pos = np.vstack([X.ravel(), Y.ravel()])          # Extracts the positions from the ordered data.      
    vals = np.vstack([init_x_vals, init_y_vals])     # Transforms the x and y values into a vertical stack for the Guassian KDE.              
    kernel = scipy.stats.gaussian_kde(vals)          # Computes Guassian KDE(Kernel Density Estimate).
    Z = np.reshape(kernel(pos).T, X.shape)           # Fits Guassian KDE values to desired coordinate system.
    
    #***Visualize Gaussian KDE.***#
    image = axs[0, 1].imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r, extent=[xmin, xmax, ymin, ymax])
    axs[0, 1].set(title='Gaussian Kernel Density Estimate.', xlabel='X Dimension (m)', ylabel='Y Dimension (m)')
    axs[0, 1].set_aspect('auto')
    fig.colorbar(image, ax=axs[0, 1])    

    #***Lognormal Distribution of Muon momentum.***#
    hist1, bins1, patches1 = axs[1, 0].hist(momentum_vals, bins=np.logspace(1, 5, no_bins), density=True, color='orange', alpha=0.5, edgecolor='k')
    bins1_centres = (bins1[1:] + bins1[:-1])/2
    pdf1 = (np.exp(-(np.log(bins1_centres) - 6.55)**2/(2*1.8**2))/(bins1_centres*1.8*np.sqrt(2*np.pi)))
    axs[1, 0].plot(bins1_centres, pdf1, linewidth=3, color='r', label='Lognormal Distribution')
    axs[1, 0].set(xscale='log', title='Lognormal Distribution of Muon momenta.', xlabel='Momentum (GeV/c)', ylabel='P(Momentum)')
    axs[1, 0].legend()
    
    residuals1 = hist1/pdf1 - 1
    axs[1, 1].hist(residuals1, bins=100, edgecolor='k')
    axs[1, 1].set(title='Residuals from Lognormal Distribution.', xlabel='Fractional difference from expected result', ylabel='Frequency')
    
    #***cos^2(θ) Distribution of Zenith Angles.***#
    hist2, bins2, patches2 = axs[2, 0].hist(theta_vals, bins=no_bins, density=True, label="Accept/reject", edgecolor='k')
    bins2_centres = (bins2[1:] + bins2[:-1])/2
    pdf2 = np.cos(bins2_centres)**2/(np.pi/4)
    axs[2, 0].plot(bins2_centres, pdf2, linewidth=3, color='r', label='Expected Distribution')
    axs[2, 0].set(title='Distribution of Zenith angles.', xlabel=r'$\theta \; (rad)$', ylabel=r'$P(\theta)$')
    axs[2, 0].legend()
    
    residuals2 = hist2/pdf2 - 1
    axs[2, 1].hist(residuals2, bins=100, edgecolor='k')
    axs[2, 1].set(title='Residuals from Expected Distribution.', xlabel='Fractional difference from expected result', ylabel='Frequency')

    #***Uniform Distribution of Azimuthial angles.***#
    hist3, bins3, patches3 = axs[3, 0].hist(phi_vals, bins=no_bins, density=True, color='orange', alpha=0.5, edgecolor='k', xunits='radians')
    bins3_centres = (bins3[1:] + bins3[:-1])/2
    pdf3 = [1/(2*np.pi) for i in range(100)]
    axs[3, 0].plot(bins3_centres, pdf3, linewidth=3, color='r')
    axs[3, 0].set(title='Uniform Distribution of Azimuthial angles.', xlabel=r'$\phi \; (rad)$', ylabel=r'$P(\phi)$')

    #***Residuals of the Uniform Distribution.***#
    residuals3 = hist3/pdf3 - 1
    axs[3, 1].hist(residuals3, bins=100, edgecolor='k')
    axs[3, 1].set(title='Residuals from Uniform Distribution.', xlabel='Fractional difference from expected result', ylabel='Frequency')

    plt.show()
    
    Residual_Analysis(residuals1, 'Lognormal Distribution Residual Analysis')
    Residual_Analysis(residuals2, 'Expected Distribution Residual Analysis')
    Residual_Analysis(residuals3, 'Uniform Distribution Residual Analysis')
    
    return None     

Plotter_to_Verify()
Charge_Ratio_Validation()

## OBJECT ORIENTED PROGRAMMING APPROACH

### Particle():

A class which includes all distributions of the variables of interest was constructed. All these distributions are stored in the `Main_Generator()` function. This function, generates a muon from all distributions of interest and returns a direction and energy. This function does not return any output but stores all values as object attributes. This is essential for the second part of this assignment. To be more clear, it enables the programme to pass a single parameter which contains all the variables required to construck the muons track through the drift chamber. 

It is important to note that since the Zenith angle distribution does not have an analytical inverse an Accept/Reject method was implemented to generate it. The Azimuthial angle distribution is uniform, meaning that `np.random.random()` is suitable for the generation of uniformly distributed values. 

It was given in the assignment the the momentum is LogNormally distributed with mean equal to 6.55 and standard deviation equal to 1.8. To generate results a Numpy routine was followed. More specifically, `np.random.lognormal(mu, sigma)` was used.

To generate the horizontal plane above the drift chamber where muons come from random numbers for the x and y coordinates where generated using Numpy. The size of the plane is 2 by 2 meters which is significantly larger than the drift chamber, allowing muons to enter from the sides as well. Demonstrators also confirmed that this is correct. The charge distribution of muons had to be generated carefully, as we are given that for cosmic ray muons there's an excess of positive muons by 30%. As a result, a random number was generated and splitted into a 1.3 : 1 ratio for every muon. The end result is a distribution of muons which each time has a 30% percent excess of positive muons. This is also validated by the `Charge_Ratio_Validation()` function.

All results except for coordinates(in S.I. units) were calculated and presented in natural units. The reason for this is that in natural units, mass, energy, momentum and charge are not close to the machine epsilon and errors are avoided.

### Testing the Distributions

To test the distributions function `Plotter_to_Verify()` was defined. This visualises the distributions by plotting the relevant histrograms for the generated variables, namely momentum, Zenith angles and Azimuthial angles. All three histograms plotted at first look behave as expected. However, to increase the confidence of the results the line of best fit was fitted into the histograms. The end result is a validation of the results.

Finally, for a much more quantitative comparison of the histrograms to the expected values the residuals of each bin where plotted, showing by what fraction the results differ from the expected values. These differences were visualised in the form of histograms as well. The mean and standard deviation of the residuals is obtained by implementing the `Residual_Analysis` function. This prints the mean and standard deviation of each residual plot.

It was crucial to ensure that the initial positions of the generated muons are evenly distributed. If not then results for the ionisation of the Argon gas in the second part of this assignment will most likely be misleading. To validate the even distribution of muons, a Gaussian Kernel Density Estimate was calculated using the `scipy.stats.gaussian_kde` method. The implementation was performed by closely consulting the Official Documentation of Scipy. The Gaussian KDE plot verifies the fact that muons are evenly distributed across the horizontal 2 dimensional plane above the drift chamber. This gives a more quantitative nature to the results, which is always more formal and raises no doubts.

### Possible Improvement - A more realistic approach to the Muons energies

Searching through literature, it was found the typical cosmic muons energies range between 4-6 GeV. In the present simulation, the energies of the generated muons range between one and two orders of magnitude higher than the literature values. This suggests that in the assignment handout, the mean and standard deviation given are not of the logarithmic distribution but of the normal. However the assignment still claims these are of the underlying lognormal distribution. For this sole reason the mean and standard deviation given were used in the lognormal distribution. 

The improvement proposed is based on the mean and standard deviation being of the normal distribution, meaning that they need to be converted in order to be placed in the parameters of the lognormal distribution. Various sources on statistics and statistical modelling agree that the formulae which can convert the mean and stardard deviation are:
$$\mu=\ln \left(\frac{\mu_{X}^{2}}{\sqrt{\mu_{X}^{2}+\sigma_{X}^{2}}}\right)$$
and
$$
\sigma^{2}=\ln \left(1+\frac{\sigma_{X}^{2}}{\mu_{X}^{2}}\right)
$$.

This is expected the generate muons with energies which are much closer to the literature value. However this would change the visualisation of the momentum distribution which itself was given to be lognormal. Under different circumstances the proposed imporvement would have been implemented. 


### Some remarks and References used for Part 1

Every histogram is within the expected distribution as the means of their residuals are very close to zero and within one standard deviation of zero. The momentum and Zenith angle distributions always exhibit a few residual outliers, always at the tail ends of the distributions, due to a very low probability. In other words they are vulnerable to random fluctuations which give rise to the relatively large residuals.

The {x, y} coordinates construct a square plane with an almost perfect even distribution as proven by the Gausian Kernel Density Estimate. The muon density is almost the same across the defined plane.

The muon charge ratio shows that cosmic rays indeed consist of 30% percent more positive muons than negative muons.


**1)Neddermeyer, S.H.; Anderson, C.D. (1937). "Note on the Nature of Cosmic-Ray Particles" (PDF). Physical Review. 51 (10): 884–886.**

**2)https://uncw.edu/phy/documents/cosmicraymuons.pdf**

**3)Rosenblatt, M. (1956). "Remarks on Some Nonparametric Estimates of a Density Function". The Annals of Mathematical Statistics. 27 (3): 832–837**

**4)Scott, D. (1979). "On optimal and data-based histograms". Biometrika. 66 (3): 605–610. doi:10.1093/biomet/66.3.605.**

**5)https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html**

## Part 2 - Ionisation

When a charged particle, such as a cosmic ray muon, traverses the wire chamber, it will ionise gas molecules. In order to model this ionisation, we can assume the muon loses negligible energy while traversing the detector, and liberates a fixed number of electrons per unit distance. For the Ar in the drift chamber we are modelling, you can assume 94 electrons are liberated per cm.  (A more sophisticated extension would use a Monte Carlo method to model variations in the number of liberated electrons as the muon traverses the chamber.)

### Writing the code

You should write a function which takes the starting position and direction of a randomly generated cosmic ray as input, and returns a 2D array representing the initial ionisation caused by the cosmic ray.  In order to achieve this, you may wish to first write a function which calculates the length of a track traversing a grid cell, given the track parameters and grid cell location. You should test your code by generating a few cosmic rays and plotting the resulting charge distribution.

In [ ]:
class Detector():
    """
    Main class containing the methods required to simulate the ionization in the drift chamber.
    Additional methods have also been developed as an extension to the original assignment
    which include the implementation of a magnetic field in the chamber, Bethe-Bloch
    enegy loss due to ionization and a Landau distribution 
    for the 94 electrons per square centimeter cell of the chamber(2D). 
    """
    
    def __init__(self, Ion_E=15.7596119):
        """
        Container for the physical constants involved in the simulation as well as
        other additional variables which are utilised throughout the simulation.
        """
        self.e = 1.60217662e-19              # Electron charge in Coulombs.
        self.c = 299792458                   # Speed of light in m/s.
        self.e_mass = 0.511e-3               # Electron mass in eV.
         
        self.density = 0.001782              # Density of Argon gas for Bethe-Bloch in g/cm^3.                   
        self.Ion_E = Ion_E*1e-9              # Ionisation energy of Argon in GeV.
        self.atomic_num = 18                 # Atomic number of Argon.
        self.atomic_mass = 39.948            # Mass number of Argon.
        self.I = 182e-9                      # Mean excitation potential for Argon for Bethe-Bloch in GeV.
        self.min_dE = 2.716215654404096e-06  # Minimum energy loss in Argon by Bethe-Bloch in GeV.
        
        self.mean_ion = 9.4e3                # Mean of total ionisations in Argon.
        self.var_e = 9.4e3                   # Poisson estimation of variance on liberated electrons equals the mean of ionisations.
        self.prim_Ionisation = 2.52e3        # Primary ionisations by muon in Argon gas at STP different from total ionisations.
    
        self.y_length = 0.5                  # Chamber dimension in meters.
        self.z_length = 0.3                  # Chamber dimension in meters.
        
        self.moyal = 'on'                    # Turns Landau Distribution of electrons on.(Landau Distribution approximated by Moyal method)
    
    def Ionisation(self, param, Grid_Cell_Width, dl, E=[0,0,0], B=[0,0,0]):
        """
        Responsible for ionizing the Argon gas in the drift chamber by placing both 
        an electric and magnetic field. The electric and magnetic 
        fields multiplied by the spatial step are in natural units.
        
        """
        self.Grid_Cell_Width = Grid_Cell_Width
        self.dl = dl                            # Spatial step size.
        self.E = np.array(E)                    # Numpy interpretation of E field array.
        self.B = np.array(B)                    # Numpy interpretation of B field array.
        
        self.Bxdl_nat_units = self.B*self.dl*3.509441707     # Conversion of B x dl to natural units. (GeV) #
        self.Exdl_nat_units = self.E*self.dl*1.170623731e-8  # Conversion of E x dl to natural units. (GeV) #
        
        param.init_pos[1] = 0  # z=0. Starting coordinate from {x,z} --> {y,z}.
                              
        Ny = int(self.y_length/Grid_Cell_Width)     
        Nz = int(self.z_length/Grid_Cell_Width)     
        self.ion_grid = np.zeros((Nz,Ny))      # Matrix Grid to fill with ionisation values.
        
        self.i = 0        
        
        self.Iterator(param)  
        
        return self.ion_grid 
           
    def Iterator(self, param):
        """
        Method allows the program to pass through the arguments of the methods of the 
        Detector(). Later on this plays a paramount role in creating 
        instances of the class comfortably without clashes or errors.
        The instances are then used to call the required methods.
        """
        
        r = [param.init_pos[0], param.init_pos[1]]  # {y,z} position of particle.  
        
        while r[0]<self.y_length and r[0]>0 and r[1]<=0 and r[1]>-self.z_length:  # While in the drift chamber. 
            
            self.i += 1
                
            Ny = int(r[0]/self.Grid_Cell_Width)       # Coordinates --> Grid Indexes.
            Nz = abs(int(r[1]/self.Grid_Cell_Width))  # Negative z, so origin is at the top of the detector.

            param.energy -= self.Bethe_Bloch(param)   # Energy loss by Bethe-Bloch.
            
            no_e = self.Freed_Electrons()             # Liberated electrons for every spatial step dl.

            self.ion_grid[Nz,Ny] += no_e*self.e       # Adding charge to the grid.

            r = r + self.dl*param.momentum_dir[1:]    # Updating to position of next iteration.
            
            if param.energy < param.mass:             # When energy of particle becomes less than its mass iteration stops.
                print('Inadequate energy.')
                break
            else:
                param.momentum_mag = np.sqrt(param.energy**2 - param.mass**2)  # Momentum of particle from energy loss. 
            
            param.momentum_mag , param.momentum_dir, param.energy = self.Lorentz_Force(param) # Applying the Lorentz force, updating momentum and energy.   
               
    def Moyal_Distribution(self, mean, var):
        """
        This is part of the extension proposed by Jim Brooke in the assignment handout.
        A Monte-Carlo method is implemented to model a Landau Distribution of the 
        94 electrons in each square centimeter cell of the drift chamber(2D).
        """        
        if self.moyal == 'on': 
            self.loc = mean - ((np.sqrt(2*var))/(np.pi))*(0.577215664901532 + np.log(2))  # Loc and Scale parameters.
            self.scale = np.sqrt(2*var)/np.pi                                             # Using the mean and variance.                               
            return moyal.rvs(loc=self.loc, scale=self.scale)                              # Random number is returned.
        else:
            return mean                                                                   # Fixed number returned if Moyal is activated. 
    
    def Freed_Electrons(self): 
        """
        Returns the liberated electrons due to ionization of the Argon gas.
        """
        return -int(self.Moyal_Distribution(self.mean_ion, self.var_e))*self.dl*(self.dE/self.min_dE)   
                                                                         
    def Lorentz_Force(self, param):
        """
        Simple setup of the Lorentz force, which acts when a magnetic field is present.
        """
        mag_v = param.momentum_mag/param.energy  # Velocity magnitude in natural units.  
       
        p = param.momentum_mag*param.momentum_dir + (param.charge)*(self.Exdl_nat_units/(mag_v) + np.cross(param.momentum_dir, self.Bxdl_nat_units))
    
        momentum_mag, momentum_dir = self.Vector_Direction_Magnitude(p)  # Direction and Magnitude of momentum.#         
        energy = np.sqrt(momentum_mag**2 + param.mass**2)                # Energy.   
        
        return momentum_mag, momentum_dir, energy               
    
    def Bethe_Bloch(self, param):
        """
        Bethe formula describes the mean energy loss per distance travelled of swift charged
        particles traversing matter (or alternatively the stopping power of the material).
        For electrons the energy loss is slightly different due to their small mass 
        and their indistinguishability. Thus relativistic corrections are required.
        Larger losses by Bremsstrahlung are suffered, so terms have been added 
        to account for this. A Landau distribution is used for the electrons.
        """        
        z = param.charge/param.e_charge_nat      # Converting electron charge from natural units to e to use in Bethe-Bloch.  
        mag_v = param.momentum_mag/param.energy  # Obtain velocity in Natural Units.
        gamma = param.energy/param.mass          # Gamma(γ) factor in natural units. 
                
        Tmax = (2*self.e_mass*((mag_v*gamma)**2))/(1 + ((2*gamma*self.e_mass)/(param.mass)) + (self.e_mass/param.mass)**2)
        ln = np.log((2*self.e_mass*((mag_v*gamma)**2)*Tmax)/(self.I**2))
        dE = 0.307075e-3*(self.atomic_num/self.atomic_mass)*((z/mag_v)**2)*(0.5*ln - mag_v**2 )*self.density*(self.dl*100)
        
        variated = self.Moyal_Distribution(dE*1e9, dE*1e9)  # Scaling energy loss to hold in the Moyal approximation. 
        self.dE = variated/1e9                              # Scaling random Moyal output back to original scale for energy loss dE.
        
        return self.dE    
        
    def Vector_Direction_Magnitude(self,p): 
        """
        This method is used to calculate and return the magnitude and direction of the 
        momentum vectors.
        """        
        momentum_mag = np.linalg.norm(p)
        momentum_dir = (1/momentum_mag)*p
        return momentum_mag, momentum_dir

In [ ]:
Ion = Detector() # Creating an instance of the Detector().

def Energy_Loss_and_Moyal_Dist_Verification():
    """
    Creates a plot of the energy loss according to Bethe Bloch as well as
    a histogram of the Moyal Distribution of the electons in the chamber
    with a line of best fit. Residuals analysis is also performed and shown
    in a histogram.
    """
    
    Muon.Main_Generator(Ion.y_length, Ion.y_length)  

    momenta = np.logspace(-3, 3, 100)                # Spacing for momentum values.
    energies = np.sqrt(momenta**2 + Muon.mass**2)    # Redefining energies.
    Ion.dl = 1e-2                                    # Ionisation steps.
    Ion.moyal = 'off'                                # Landau/Moyal Distribution.
    dE = []                                          # Store energy values.
    Ionisation_loss = []                             # Store values for energy loss due to ionisation.
    
    for i in range(100):
        Muon.energy = energies[i]
        Muon.momentum_mag = momenta[i]
        dE.append(Ion.Bethe_Bloch(Muon))   
    
    min_dE = np.amin(dE[20:])   

    for i in range(100):
        Ionisation_loss.append((Ion.prim_Ionisation/100)*(dE[i]/min_dE)*Ion.Ion_E) 
        
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5), dpi=150)  
    
    #***Plotting Bethe-Bloch energy loss and energy loss to primary ionisation.***#
    ax1.plot(momenta, dE, color='b', label='Bethe Bloch')
    ax1.plot(momenta, Ionisation_loss, label='Loss due to primary Ionisation')
    ax1.set(title='Energy loss due to Ionisation.', xscale='log', yscale='log', xlabel='Momentum (GeV/c)', ylabel='dE/dx (GeV/cm)')
    ax1.legend()
    
    Ion.moyal = 'on' 
    electrons_liberated = [Ion.Moyal_Distribution(9400,9400) for i in range(10000)] 
    
    #***Plotting Landau/Moyal Distribution of electrons per cell.***#                                                              
    hist, bins, patches = ax2.hist(electrons_liberated, bins=100 ,density=True, edgecolor='k', label='Generated electrons per cell.')
    bins_centres = (bins[1:] + bins[:-1])/2
    pdf = moyal.pdf(bins_centres, Ion.loc, Ion.scale)
    ax2.plot(bins_centres, pdf, color='r', linewidth=3, label='Moyal Distribution of electrons per cell.')
    ax2.set(title='Moyal Distribution of electrons in each cell.', xlabel='Liberated electrons per meter', ylabel='Normalized Frequency')
    ax2.legend()
    
    residuals = hist/pdf - 1
    ax3.hist(residuals, bins=100, edgecolor='k')
    ax3.set(title='Residuals from Moyal Distribution.')
    
    Residual_Analysis(residuals, 'Moyal Distribution of electrons residuals')
    
Energy_Loss_and_Moyal_Dist_Verification()

In [ ]:
def IonisationPlot(Ionisation, particle, Grid_Cell_Width=0.001, dl=0.0002, E=[0,0,0], B=[0,0,0], rows=2, cols=3):
    """
    This function will show images of the ionization track inside the drift chamber for different  ionizations.
    The ionization for any given electric or magnetic field can be shown. However, fixed electric and magnetic 
    fields have been chosen to allow the demonstrator to assess the code, as instructed by Dr. Jim Brooke.
    """    
    axes=[]
    fig=plt.figure(figsize = (24,10)) 

    for a in range(rows*cols):   

        particle.Main_Generator(Ionisation.y_length, Ionisation.y_length) 
        energy = np.copy(particle.energy)  
        image = Ion.Ionisation(particle, Grid_Cell_Width=Grid_Cell_Width, dl=dl, E=E, B=B) 

        axes.append(fig.add_subplot(rows, cols, a+1))  
        subplot_title=('Ionisation Grid with Landau/Moyal Distribution.')
        axes[-1].set_title(subplot_title)
        axes[-1].set(xlabel='Z dimension (mm)', ylabel='Y dimension (mm)')
        
        im = plt.imshow(image, cmap='jet')
        color_bar = plt.colorbar(im, shrink=0.8)
        color_bar.set_label('C/mm^2')
                             
E=[1e5,1e5,1e5]  # Setting E field.
B=[1e4,0,-1e4]   # Setting B field. 
   
IonisationPlot(Ion, Muon, E=E, B=B)

### Detector Class

This class produces the output of a matrix which gives the charge distribution induced by the track of the muons inside the drift chamber. Once again, the object oriented programming approach was followed as it made the implementation of multiple functions as methods to incorporate a number of extensions. Extensions include the implementation of magnetic fields in the chamber, energy loss due to primary ionisation and Bethe-Bloch energy loss. Moreover, OOP allows for a more clear, compact and concise code, in which many arguments can be passed through the methods without clashes.

#### Ionisation - The approach

First of the equation describing ionisation must be implemented in a vectorised form, such that the output is a matrix with the charge density at each position. The vectorised equation for this is :
$$ \vec{r_{n+1}} = \vec{r_{n}} + \vec{v}dt $$
It makes more sense physically to show the position of the particle in space rather than time. Thus, timesteps had to be  converted to spatial steps along the ionisation the path. The conversion of a timestep to spatial step is given by: 
$$dt = \frac{dl}{v}$$ 
Finally the position is given by:
$$ \vec{r_{n+1}} = \vec{r_{n}} + \vec{v}\frac{dl}{v} $$
which is dependent on $\frac{\vec{v}}{v}$. In other words, only the direction of the particle, ensuring a constant accuracy in the prediction of position of the muons regardless of their incoming momentum. This method also takes into account the presence of a magnetic field in the chamber.

To ensure than the visualisation of the ionisation in the Argon gas is accurate and of high resolution, the spatial step is set to be considerably smaller than the gridsize of the image. More specifically, the length of each line in one grid of the image is computed to a high degree of accuracy. The final result of the ionisation track is visualized with a high level of confidence.

#### Magnetic Field and Lorentz Force 

The generated muons move at approximately the speed of light so relativistic Lorentz Force was required given by the vectorised equation:
$$ \vec{p_{n+1}} = \vec{p_n} + q( \vec{E} + \vec{v}x\vec{B})\frac{dl}{v}$$
where $\vec{p} = \gamma m v$ is the relativistic momentum. Momentum values are updated at each step. The process is exactly the same as in the ionisation approach described above.

#### Bethe - Bloch and Energy Loss

The Bethe-Bloch formula describes the mean energy loss per distance travelled of swift charged particles traversing matter. Electrons have a different energy loss due to their small mass which requires relativistic corrections. Moreover, they are subject to larger losses due to Bremsstrahlung, so terms must be added to account for this. Fast charged particles moving through matter interact with the electrons of atoms in the material, in this case Argon gas. The interaction excites or ionizes the atoms, leading to an energy loss of the travelling particle. The energy lost as muons traverse through is given by:
$$
-\left\langle\frac{d E}{d x}\right\rangle=K z^{2} \frac{Z}{A} \frac{\rho}{\beta^{2}}\left(\frac{1}{2} \ln \frac{2 m_{e} \beta^{2} \gamma^{2} T_{\max }}{I^{2}}-\beta^{2}\right)
$$
where K is a constant, Z is the atomic number of the Argon gas, A its mass number, z the electric charge in electron charge units, I the mean excitation potential, ρ the density of Argon gas in the chamber and $T_{max}$ the maximum kinetic energy transfered in a collision:
$$T_{\max }=\frac{2 m_{e} \beta^{2} \gamma^{2}}{1+2 \gamma m_{e} / M+\left(m_{e} / M\right)^{2}}$$
where M is the mass of the muon(incident particle).

#### Landau Distribution by Moyal Approximation

The number of ionised electrons and the energy loss follow the Landau distribution, which is approximated by the Moyal distribution(as Dr. James Brooke pointed out) given by:
$$ f(x) = \frac{\exp ( \frac{-x + e^{-x}}{2})}{\sqrt{2 \pi}} $$
This approximation is accurate for more than 5 ionisations per centimeter. For Argon there are 24.3 primary ionisations per centimeter, rendering the Moyal approximation appropriate. `scipy.moyal.rvs` was used to generate a random Moyal distribution with a certain mean and variance. This method needs to take a location and scale parameter which were converted from mean and variance following: 
$$ \bar{x} = \mu + \sigma (\text{EulerGamma} + \ln (2)) $$
$$ \text{var} = \frac{\pi^2 \sigma^2}{2} $$.
These equations and the general method are available through the official documentation of the Scipy library. 
Since, no literature value for the variace of ionisations or energy loss was available, the most accurate estimate was produced using Poisson variance(again consulting Dr. James Brooke).

#### Testing the Distributions for Part 2

Functions `Bethe-Bloch` and `Energy_Loss_and_Moyal_Dist_Verification` where tested to ensure the distributions are correct.
The Bethe-Bloch energy loss curve shows an energy loss of a few GeV which is too low, so the chance of a muon losing all of its kinetic energy in the chamber is extremely low.

Energy loss due to Bethe-Bloch is greater than the energy lost due to primary ionisation. This is expected as muons ionise electrons with with a high kinetic energy. Primary ionisations in Argon account for 33.3 percent of the total ionisations. As a result, at the minimum energy lost, 25.2 electrons per centimeter are primarily ionised. The total is 94 per centimeter.

Primary ionisations are proportional to the energy lost by the Bethe Bloch in the present case. Thus the minimum energy loss is determined in order to scale primary ionisations with the energy loss. It is assumed that total ionisations are proportional to the primary ionisations. `Freed Electrons` method take care of this.

Finally, the Moyal approximation of the Landau distribution confirms the desired curve. Residuals plots and their mean and standard deviation also confirm the fit.

#### Comments on the Ionisation Tracks

In the presence of an electric field only the ionisation tracks would be straight lines. However, this is not a very interesting topic. The magnetic field added gives rise to much more interesting ionisation tracks. The tracks are curved due to the Lorentz force. Sometimes less energetic muons which suffer from the Lorentz force will continue to follow curved tracks with a decreasing radius of curvature as their kinetic energy is decreasing due to the effect of the Lorentz force. This why sometimes a conical shape track is observed. The magnetic and electric fields have been defined as 3D vectors and tweaked such that the effect of the magnetic field on the tracks is high.

#### References used for Part 2

**1)https://web.physik.rwth-aachen.de/~hebbeker/theses/mameghani_diploma.pdf**

**2)https://en.wikipedia.org/wiki/Bethe_formula**

**3)https://meroli.web.cern.ch/lecture_landau_ionizing_particle.html**

**4)https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.moyal.html**

**5)https://en.wikipedia.org/wiki/Classical_electromagnetism_and_special_relativity**

# Part 3 - Drift-Diffusion

The charge produced by ionisation will drift towards the anode wires, where it can be detected.  However, it will also diffuse in the gas.  These two 'transport' processes are described by the drift-diffusion equation  :

$$\frac{\partial q}{\partial t} = D \nabla^2 q - \frac{1}{\mu} {\bf E} \cdot (\nabla q)$$

Here, $q(x,y)$ represents the charge distribution in 2D.  The first term on the RHS represents diffusion, the second represents the charge drift under the influence of the electric field.

By expanding in terms of (x,y) this can be written :

$$\frac{\partial q}{\partial t} = D\frac{\partial^2 q}{\partial x^2} + D\frac{\partial^2 q}{\partial y^2} - \frac{E_x}{\mu}\frac{\partial q}{\partial x} - \frac{E_x}{\mu}\frac{\partial q}{\partial y}$$

For the diffusion terms, we will use the same implicit finite difference scheme described in lectures and the previous assignment, but expanded to 2D.  For the drift terms, we will use the 1st order backward difference, also known as the "upwind" scheme (since the backward difference is "upwind" of the electrostatic force direction).  There are other choices (eg. the 2nd order centred difference) but these are prone to non-physical oscillations.

The full finite difference equation is then :

$$\frac{q^{n+1}_{i,j} - q^n_{i,j}}{\tau} = D\frac{q^{n+1}_{i-1,j} - 2q^{n+1}_{i,j} + q^{n+1}_{i+1,j}}{h^2} + D\frac{q^{n+1}_{i,j-1} - 2q^{n+1}_{i,j} + q^{n+1}_{i,j+1}}{h^2} - \frac{E_x}{\mu}\frac{(q^{n+1}_{i,j}- q^{n+1}_{i-1,j})}{2h} - \frac{E_y}{\mu}\frac{(q^{n+1}_{i,j}-q^{n+1}_{i,j-1})}{2h}$$

But we can simplify this by assuming the electric field is uniform and non-zero in the x-direction only :

$$q^n_{i,j} = - \alpha q^{n+1}_{i-1,j} - \alpha q^{n+1}_{i+1,j} - \alpha q^{n+1}_{i,j-1} - \alpha q^{n+1}_{i,j+1} + (1+4\alpha) q^{n+1}_{i,j} - \beta q^{n+1}_{i-1,j} + \beta q^{n+1}_{i,j}$$

where $\alpha = \frac{D \tau}{h^2}$ and $\beta = \frac{\tau E}{2\mu h}$.

Or collecting terms relating to the same node :
$$q^n_{i,j} = ( - \alpha- \beta) q^{n+1}_{i-1,j} - \alpha q^{n+1}_{i+1,j} - \alpha q^{n+1}_{i,j-1} - \alpha q^{n+1}_{i,j+1} + (1+4\alpha+\beta) q^{n+1}_{i,j}$$

This is an implicit equation, expressing $q^n$ in terms of $q^{n+1}$ for a node and its neighbours. Just as for the 1D diffusion problem, we will need to solve the equation for each timestep, and iterate.

And again just like the 1D diffusion problem, we can represent this finite difference equation as a matrix.  However, there is an added complication because we now have a 2D grid.  What we have to do is represent each node in the 2D grid systematically as a row (or column) of the matrix.  This means if we have a grid of size $n_i \times n_j$, the matrix will need to be $n_i n_j \times n_i n_j$.  One way of systematically mapping nodes onto rows/columns of the matrix is to use "row-major ordering", where the matrix index $k$ corresponding to grid node $(i,j)$ is $k = n_ij + i$.

Assuming the 2D grid is stored in a row-major ordered vector, then we can write the finite difference equation as a matrix equation :
$$q^n = M q^{n+1}$$

For a small example, if $n_i=n_j=4$, then matrix M is $16 \times 16$ and its top-left quadrant ($8 \times 8$) is  :

$$M = \pmatrix{
1+4\alpha+\beta & -\alpha &  &  & -\alpha  &  &  &  & \\
-\alpha-\beta & 1+4\alpha+\beta & -\alpha &  &  & -\alpha &  &  & \\
 & -\alpha-\beta & 1+4\alpha+\beta & -\alpha &  &  & -\alpha &  & \\
 &  & -\alpha-\beta & 1+4\alpha+\beta & -\alpha &  &  & -\alpha & \\
-\alpha &  &  & -\alpha-\beta & 1+4\alpha+\beta & -\alpha &  &  & \\  
  & -\alpha &  &  & -\alpha-\beta & 1+4\alpha+\beta & -\alpha &  & \\  
  &  & -\alpha &  &  & -\alpha-\beta & 1+4\alpha+\beta & -\alpha & \\
  &  &  & -\alpha &  &  & -\alpha-\beta & 1+4\alpha+\beta & \\
}$$

And the diagonals continue into the remaining quadrants.  This is a "tri-diagonal matrix with fringes".  It comprises five diagonal elements corresponding to the five terms in the previous equation.  The tri-diagonal part corresponds to the terms for $(i-1, j)$, $(i,j)$ and $(i+1, j)$ while the fringes are the terms for $(i, j-1)$ and $(i, j+1)$.  In terms of the matrix coordinate $k$, the fringe diagonals start at $(0,n_i)$ and $(n_i,0)$.

In the absence of additional terms, the matrix equation above possesses a "periodic boundary condition".  Effectively, this means any charge leaving the grid on one edge will re-appear on the opposite edge, which is adequate for our purposes here.

### Writing the code

Solving the 2D drift-diffusion equation is conceptually similar to the 1D diffusion equatino you have already encountered.  The differences are : the changes needed to handle a 2D grid of nodes, and the $\beta$ term in the matrix M to accomodate charge drift.

First you will need to write a function that will create the matrix M. You should verify that this function produces the desired matrix for a simple test case.

Next,  write a function which iteratively solves the matrix equation $q^n = M q^{n+1}$ for each timestep, using an appropriate routine from scipy.  This function should take a 2D array of the charge distribtuion as input. However, you will need to convert this to a 1D array in order to solve the matrix equation.  **Hint** : You can use `ndarray.flatten()` to convert a 2D array into a row-major ordered 1D array, and you can use `ndarray.reshape(ny, nx)` to convert a 1D row-major ordered array back into a 2D array.

You should test your code with a simple test case, eg. an initial point charge.  You may wish to also examine the behaviour under 'diffusion only' and 'drift only' conditions, ie. with E or D set to zero.

As stated in the introduction, for Argon at NTP you can assume $D=0.1$, $\mu=50 \: {\rm m^2V^{-1}s^{-1}}$ and $E=10^5 \: {\rm Vm}^{-1}$. Given these values, an appropriate run time is $\mathcal{O}(10^{-4}) \: {\rm s}$.    

In [ ]:
def Sparse_Matrix_Construction(diag, off_diag1, off_diag2, off_diag3, Ny, Nz): 
    """
    Creates and returns a sparse matrix using diagonal inputs.The sparse matrix 
    methods from SciPy are used to improve the computational efficiency in
    order to avoid unnecessary calculations where possible.
    """
    off_diag1_l = np.full(Ny*Nz, off_diag1)
    off_diag1_l[Ny-1::Ny] = 0                 # Set every Ny = zero. Removing periodic boundary conditions.
             
    off_diag2_l = np.full(Ny*Nz, off_diag2)
    off_diag2_l[Ny-1::Ny] = 0                 # Set every Ny = zero. Removing periodic boundary conditions.
              
    return scipy.sparse.diags(diagonals = [diag, off_diag1_l, off_diag2_l, off_diag3, off_diag2],
                              offsets = [0,-1,1,-Ny,Ny], shape = [Ny*Nz,Ny*Nz],
                              format = 'csc')  # CSC format for Sparse Solving Algorithm.
    
def Matrix_Generator(h, dt, y_length, z_length, D=0.1, mu=50, E=[0,1e5,0]): 
    """
    Accepts the required arguments and gives the diogonals to Sparse_Matrix_Construction.
    """    
    a = D*dt/(h**2)
    b_y = dt*E[1]/(2*mu*h)
    b_z = dt*E[2]/(2*mu*h)
    Ny = int(y_length/h)     
    Nz = int(z_length/h)
    
    diag = 1 + 4*a + b_y + b_z   
    off_diag1 = - a - b_y 
    off_diag2 = - a
    off_diag3 = - a - b_z
    
    M = Sparse_Matrix_Construction(diag, off_diag1, off_diag2, off_diag3, Ny, Nz)
    
    return M 

def Compute(ion_grid, h, dt, y_length, z_length, D=0.1, mu=50, E=[0,1e5,0], iterations=500):  
    """
    Performs all of the required calculations using the pre-defined functions above 
    and stores the data which will be used to visualize the drift/drift-diffusion
    in the drift chamber.
    """
    q = ion_grid.flatten()     
    M = Matrix_Generator(h, dt, y_length, z_length, D, mu, E)           
    solve = factorized(M)
    currents = []
    
    for i in range(iterations):
        q = solve(q)
        currents.append(q[int(y_length/h)-1::int(y_length/h)])
    
    q_end = q.reshape(int(z_length/h), int(y_length/h))    
    
    return q_end, currents              

In [ ]:
y_length = 0.1
z_length = 0.1  

h = 0.001     
dt = 1e-7  
Ny = int(y_length/h)  
Nz = int(z_length/h)

ion_grid = np.zeros((Ny,Nz)) 

ion_grid[50,10] = 1.6e-19 

#***Creating image data for the Drift-Diffusion Tests.***#

diff, a1  = Compute(ion_grid, h, dt, y_length, z_length, E=[0,0,0])  
drift, a2 = Compute(ion_grid, h, dt, y_length, z_length, D=0)    
q, a3     = Compute(ion_grid, h, dt, y_length, z_length)  
    
drift_down, a4      = Compute(ion_grid, h, dt, y_length, z_length, E=[0,0,1e5], D=0)  
drift_diff_down, a5 = Compute(ion_grid, h, dt, y_length, z_length, E=[0,0,1e5])  
drift_diag, a6      = Compute(ion_grid, h, dt, y_length, z_length, E=[0,1e5,1e5], D=0) 
drift_diff_diag, a7 = Compute(ion_grid, h, dt, y_length, z_length, E=[0,1e5,1e5])  

In [ ]:
def Drift_Diffusion_Plotter(a, b, c, d, titles):
    """
    Takes the arrays of data stored in the releavnt lists created by Compute and 
    creates images of the drift and drift diffusion using Matplolib imshow.
    """    
    fig, axs = plt.subplots(1, 4, figsize=(20, 10), dpi=150)
    
    im = axs[0].imshow(a)
    axs[1].imshow(b)
    axs[2].imshow(c)
    axs[3].imshow(d)
    color_bar = plt.colorbar(im, ax=axs.ravel().tolist(), shrink = 0.35)
    color_bar.set_label('C/mm^2')
    
    for i, ax in enumerate(axs.flatten()):
        ax.set_title(f'{titles[i]}')
        
    return None

def Current_Plotter(currents, h=0.001, t=1e-7, t_steps=500):
    charge = []
    charge.append(np.sum(currents[0]))

    for i in range(1, t_steps):
        charge.append(np.sum(currents[i]) + charge[-1])

    current = -(np.array(charge[1:]) - np.array(charge[:-1])) / t

    t_vals = np.linspace(0, t * t_steps, len(current))

    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)

    ax.plot(t_vals, current, color='blue')
    ax.set(xlabel='Time (s)', ylabel='Current (A)')

    plt.show()
    
    return None

In [ ]:
titles1 = ['Initial Charge', 'Diffusion', 'Drift', 'Drift-Diffusion']  
titles2 = ['Drift Downwards', 'Drift-Diffusion Downwards', 'Diagonal Drift', 'Diagonal Drift-Diffusion']

Drift_Diffusion_Plotter(ion_grid, diff, drift, q, titles1) 
Drift_Diffusion_Plotter(drift_down, drift_diff_down, drift_diag, drift_diff_diag, titles2)


### Why use a sparse matrix?

A sparse matrix is generated using the `Sparse_Matrix_Construction` and `Matrix_Generator` functions. There are many zeros and the matrix of size $N_y*N_z$ is very large. A sparse matrix algorithm is less dense than other conventional matrix solving methods and considerably faster. The 'csc' format is used as it is the most compatible with this method according to official documentation.

In order to eliminate periodic boundary conditions, it was demanded that elements from the edges of the grids do not interfere with each other. To achieve this, the grid dimensions were set as $N_y \times N_z$ for every element in $N_y$ which is then removed from the off-diagonals near the centre of the matrix. This removes any periodic boundary conditions and makes the whole procedure more straightforward.

`Compute` flattens the relevant array and implements `scipy.sparse.linalg.factorized` to factorise the matrix using LU decomposition outside the iterative loop, finally solving the matrix equation inside the iterative loop. According to the official documentation this is the fastest method to execute these computationally demanding calculations, especially when using a 'csc' format.

#### Results for Drift - Diffusion ( This is an initial test to ensure the algorithm operates as required. )

As required the algorithm is first tested for a point charge and the effects of drift, diffusion and drift-diffusion in different directions where visualised.

1) Diffusion - Isotropic spreading out of charge.

2) Drift - Charge drifts to right and is spread out by the EM field.

3) Drift-Diffusion - As the name suggests, combine both of the effects.

4) Downward Drift - Testing a different direction of the drift by tweaking the EM field.

5) Downward Drift-Diffusion - Incorporating diffusion to 4). 

6) Diagonal Drift - Testing the drift in a combination of directions(downwards and to the right).  

7) Diagonal Drift-Diffusion - Incorporating diffusion to 6).


The images plotted above confirm that algorithm functions properly and gives rise to accurate physical modelling. 
It is also confirmed that the periodic boundary conditions have been removed successfully as charge does not pass to the other side.

#### Important note for the 7th plot of diagonal drift.

It is a bit concerning that in the 7th plot we see drift diffusion instead of just diagonal drift. There is an explanation of this. 

Thinking about it in depth, one can figure out that inducing drift in 2 directions will inevitably spread out the charge distribution, producing a diffusion like image. This happens because the grid is constructed such that there are only horizontal and vertical directions. There are not any diagonal directions. Physically, there is no spreading of the charge in diagonal diffusion, this is an effect of the simulation worth noting. 

#### References for Part 3

**1)https://en.wikipedia.org/wiki/Diffusion_current**

**2)F. Sauli (1977), - Principles of operation of multiwire proportional and drift chambers Retrieved 2012-02-25**

**3)https://web.archive.org/web/20150213145706/http://www.kip.uni-heidelberg.de/~coulon/Lectures/Detectors/Free_PDFs/Lecture8.pdf**

**4)Golub, Gene H.; Van Loan, Charles F. (1996). Matrix Computations (3rd ed.). Baltimore: Johns Hopkins. ISBN 978-0-8018-5414-9.**

**5)Stoer, Josef; Bulirsch, Roland (2002). Introduction to Numerical Analysis (3rd ed.). Berlin, New York: Springer-Verlag. ISBN 978-0-387-95452-3.**

# Part 4 - Simulating the Drift Chamber

Now use the functions you have defined above to simulate the full process :
* a cosmic muon entering the drift chamber
* the initial charge distribution resulting from that cosmic muon
* the charge drifting, and diffusing, under the influence of the electric field

Run these steps for at least one cosmic ray muon, and plot the results in an appropriate format.

In [ ]:
Muon = Particle()    
Ion = Detector() 

h = 0.002     
      
#***Generating Ionisation Patterns for Drift-Diffusion with E and B field present.***#
Muon.Main_Generator(Ion.y_length, Ion.z_length)   
Ion_grid = Ion.Ionisation(Muon, Grid_Cell_Width=h, dl=(h/5), E=[1e5,1e5,1e5], B=[1000,0,-1000]) 

dt = 1e-7
q_end,  a8  = Compute(Ion_grid, h, dt,  Ion.y_length, Ion.z_length)  # Drift-Diffusion Stage 1.#
dt = 5e-7
q_end2, a9  = Compute(Ion_grid, h, dt, Ion.y_length, Ion.z_length)   # Drift-Diffusion Stage 2.#
dt = 9e-7
q_end3, a10 = Compute(Ion_grid, h, dt, Ion.y_length, Ion.z_length)   # Drift-Diffusion Stage 3.#

titles3 = ['Initial Charge.', 'Drift-Diffusion Stage 1.', 'Drift-Diffusion Stage 2.', 'Drift-Diffusion Stage 3.']


Drift_Diffusion_Plotter(Ion_grid, q_end, q_end2, q_end3, titles3)

In [ ]:
Current_Plotter(a10)

## END RESULTS - FINAL GOAL OF THE ASSIGNMENT

Combining classes `Particle` and `Detector` from Parts 1 and 2 respectively, as well as the functions from Part 3 the following simulation has been successfully performed:

A random muon out of all the generated muons enters the drift chamber at random entry point, which is either from the top or the sides and then exits. Due to magnetic field being present along the electric field, the entry and exit points are not connected by a straight line. As long as the muon is inside the chamber it transfers its kinetic energy(which is modelled using Bethe-Bloch) to the Argon atom electrons, causing them to leave the atom, in other words ionisation takes place. Incorporating both the `Particle Class` and Part 3 functions the drift-diffusion was successfully modelled as shown above including a magnetic field.

#### Current 

The current along a random wire at the end of the chamber was also visualised using the `Current_Plotter` function. It is important to note that the visualisation of the current is only possible if the drift-diffusion is allowed to reach the end of the chamber. To do that the simulation is allowed to run for an appropriate time and the drift diffusion is visualised at different stages. The current is plotted is always using the data from last image of the drift diffusion, so if the drift diffusion in the last picture has not almost fully passed through the RHS of the image the current will have not reach zero again.